# xdggs example to prepare H3 dataset

In [ ]:
import h3
import h3.api.numpy_int
import h3.unstable.vect
import numpy as np
import xarray as xr

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature").load()
ds

In [ ]:
ds.air[0].plot()

In [ ]:
resolution = 3

lon, lat = xr.broadcast(ds.lon, ds.lat)

In [ ]:
%%time
index = h3.unstable.vect.geo_to_h3(lat.data.ravel(), lon.data.ravel(), resolution)

In [ ]:
index.shape = lon.shape

len(np.unique(index)) / lon.size

In [ ]:
index.shape

In [ ]:
ds.lon.shape

In [ ]:
ds.coords["index"] = ("lat", "lon"), index.transpose()
ds

In [ ]:
ds.index.plot()

In [ ]:
lon_min, lon_max = ds.lon.min().values.item(), ds.lon.max().values.item()
lat_min, lat_max = ds.lat.min().values.item(), ds.lat.max().values.item()

In [ ]:
import shapely

bbox_coords = [
    (lon_min - 360, lat_min),
    (lon_min - 360, lat_max),
    (lon_max - 360, lat_max),
    (lon_max - 360, lat_min),
    (lon_min - 360, lat_min),
]
bbox = shapely.Polygon(bbox_coords)
bbox

In [ ]:
bbox_coords

In [ ]:
# h3 wants lat first
bbox_coords_lat_first = [(lat, lon) for lon, lat in bbox_coords]
bbox_indexes = np.array(
    list(h3.api.basic_int.polyfill_polygon(bbox_coords_lat_first, resolution))
)
bbox_indexes.shape

In [ ]:
ll_points = np.array([h3.api.numpy_int.h3_to_geo(i) for i in bbox_indexes])
ll_points_lon_first = ll_points[:, ::-1]

In [ ]:
coords = {"cell_ids": bbox_indexes}

# remember to re-add the 360 degree offset
dsi = ds.interp(
    lon=xr.DataArray(ll_points_lon_first[:, 0] + 360, dims="cell_ids", coords=coords),
    lat=xr.DataArray(ll_points_lon_first[:, 1], dims="cell_ids", coords=coords),
)
dsi

In [ ]:
dsi2 = dsi.drop_vars(["lon", "lat", "index"])
dsi2.cell_ids.attrs = {"grid_name": "h3", "resolution": resolution}
dsi2.to_netcdf("data/h3.nc", mode="w")
dsi2